<a href="https://colab.research.google.com/github/dayviddouglas/ciencia_de_dados_licitacoes_prefeitura_do_recife/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install dash
!pip install plotly-express

In [12]:
import pandas as pd
import requests as req
from io import StringIO
import urllib
import matplotlib.pyplot as plt
import dash as dsh
from dash import dcc, html,dcc
import plotly.express as px

file_id= "14NuFHR-tJe0xAv_FTIJrM9-6u7f49cJr"
url =  f"https://drive.google.com/uc?id={file_id}"


try:
    response = req.get(url)
    response.encoding = 'utf-8'
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    base = pd.read_csv(csv_raw, sep=';', encoding='utf-8')



except req.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")




In [13]:
base

,"<!doctype html><html lang=""en-US"" dir=""ltr""><head><base href=""https://accounts.google.com/v3/signin/""><link ref=""preconnect"" href=""//www.gstatic.com""><meta name=""referrer"" content=""origin""><style data-href=""https://www.gstatic.com/_/mss/boq-identity/_/ss/k=boq-identity.AccountsSignInUi.LOjzvKtZM0I.L.W.O/am=PwBGokOOAlCAIwQNABAAKAAAAAAAAAAABDLAJAQ/d=1/ed=1/rs=AOaEmlFs23JX9SZ7cYchpEP_Tnu-f3aCrA/m=identifierview,_b,_tp"" nonce=""zvwhZZDss1UYIHYXE_edAA"">c-wiz{contain:style}c-wiz>c-data{display:none}c-wiz.rETSD{contain:none}c-wiz.Ubi8Z{contain:layout style}@-webkit-keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2)","-webkit-transform:translate(-50%,-50%) scale(0.2)","transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:translate(-50%,-50%) scale(2.2)","-webkit-transform:translate(-50%,-50%) scale(2.2)","transform:translate(-50%,-50%) scale(2.2)}}@keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2)","-webkit-transform:translate(-50%,-50%) scale(0.2).1","transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:translate(-50%,-50%) scale(2.2).1","-webkit-transform:translate(-50%,-50%) scale(2.2).1","transform:translate(-50%,-50%) scale(2.2)}}@-webkit-keyframes quantumWizIconFocusPulse{0%{-webkit-transform:translate(-50%,-50%) scale(1.5)","-webkit-transform:translate(-50%,-50%) scale(1.5)",...,ifkv=AS5LTAT9z-h_AH9Ps0WNgrQruRM1B67re9dsO5XiBIV_bIBeE0HTMHrWwi6vQ3BIznGqBCkhyjLlZw&amp.1,"service=wise"" method=""POST"" novalidate><div class=""iEhbme"" jsname=""rEuO1b""><section class=""aN1Vld ""><div class=""yOnVIb"" jsname=""MZArnb""><div class=""Fu5aXd"" jsname=""dWPKW""><div class=""Flfooc""><div class=""TRuRhd YKooDc""><div class=""fjpXlc""><label class=""dXXNOd""><input class=""xyezD"" jsname=""Ufn6O"" type=""email"" name=""identifier"" id=""identifierId"" autofocus autocapitalize=""none"" autocomplete=""username"" dir=""ltr""/><div class=""nWPx2e""><div class=""YhhY8""></div><div class=""CCQ94b"">Email or phone</div><div class=""tNASEf""></div></div></label></div></div></div><div class=""F3wxlc"" jsname=""h9d3hd""></div><div class=""NHVGlc"" jsname=""JIbuQc""></div></div><p class=""vOZun"" jsname=""OZNMeb"" aria-live=""assertive""></p><p class=""vOZun""><a href=""/signin/usernamerecovery?continue=https://drive.usercontent.google.com/download?id%3D14NuFHR-tJe0xAv_FTIJrM9-6u7f49cJr&amp",dsh=S-452528290:1718048762527654&amp.2,flowEntry=ServiceLogin&amp.2,flowName=WebLiteSignIn&amp.2,followup=https://drive.usercontent.google.com/download?id%3D14NuFHR-tJe0xAv_FTIJrM9-6u7f49cJr&amp.2,ifkv=AS5LTAT9z-h_AH9Ps0WNgrQruRM1B67re9dsO5XiBIV_bIBeE0HTMHrWwi6vQ3BIznGqBCkhyjLlZw&amp.2,"service=wise"" jsname=""Cuz2Ue"">Forgot email?</a></p><input type=""password"" name=""hiddenPassword"" class=""hJIRO"" tabindex=""-1"" aria-hidden=""true"" spellcheck=""false"" jsname=""RHeR4d""><input type=""hidden"" name=""usi"" value=""S-452528290:1718048762527654""><input type=""hidden"" name=""domain"" value=""""><input type=""hidden"" name=""region"" value=""US""><input type=""hidden"" name="""" value="""" jsname=""duMqid"" id=""fidoUserHandle""><span jsname=""xdJtEf""><script nonce=""-NGYn2twCvS1MyoRzr0jIw"">//# sourceMappingURL=data:application/json",charset=utf-8,"base64,eyJ2ZXJzaW9uIjogMywic291cmNlcyI6WyIiXSwic291cmNlc0NvbnRlbnQiOlsiICJdLCJuYW1lcyI6WyJjbG9zdXJlRHluYW1pY0J1dHRvbiJdLCJtYXBwaW5ncyI6IkFBQUE7QUFBQTtBQUFBO0FBQUE7QUFBQTtBQUFBO0FBQUEifQ=="
0,"(function(){var U=function(u,L,e,b,Z,F,Y,J,M,O...",27!=l,")if(80==l)S=function(){},F=void 0,b=h(u,functi...",50!=R,")11==R?R=S?77:50:77==R&&(e&&p(e),F=c,S(),S=voi...",75!=k,"){if(74==k)return w=Y(t),c&&c(w),w",8==k?(x=function(){F(function(W){p(function(){...,else if(10==l)l=76,else{if(79==l)return X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"'(function(){/*',",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"'',",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,